In [38]:
import numpy as np
import Elementos2D as BL4
import sympy as sp
import meshio
import Malha2D

In [39]:
#Entrada de Dados 
#Nesses Dics devemos ter os nomes das propriedades iguais dos nossos Physical Groups
Materiais = {"MAT1":[1e9,0.3]}
#Nome do Apoio(igual do Physical Group:"direção a restringir o movimento(x,y ou engaste(ambos))")
apoios = {"Engaste1":"y","Engaste2":"x"}
#Carregamento em Face: "Nome da Carga(igual do Physical Group", direção:1= normal, 2 = tangencial,valor]
cargas = [("CARGA",1,1000)]

arquivo = "chapaTracao.msh"

#Gera Malha
Malha2D.Gerar_chapa_tracao_Quarto("ChapaTracao.msh",L=1.0,R=0.05,p_fino=0.01,p_grosso=0.05)

#Retorna as propriedades
nn, XY, ne, IJ, MAT, ESP, na, AP, nc, P = Malha2D.Processar_malha2D(arquivo,Materiais,apoios,cargas,espessura=0.01)


In [40]:
#Monta Rigidez Global
K_b = BL4.RigidezGlobal(nn,ne,MAT,ESP,XY,IJ)
#Monta Força Global
F_b = BL4.ForcapGlobal(nn,ESP,XY,IJ,nc,P)
#Aplica as CChs homogêneas
K_mod,F_mod = BL4.AplicaCCH(nn,na,AP,K_b,F_b)

#Calcula o Deslocamento
U = np.linalg.solve(K_mod,F_mod)

detJ = BL4.CalculaDetJ(ne,XY,IJ)

#Calcula a Tensão ao longo dos elementos
Sigma = BL4.CalculaTensaoMalha(nn,ne,MAT,ESP,XY,IJ,U)

#Calcula Von Mises
TVM = BL4.TensaoVonMises2D(ne,Sigma)

In [41]:
#Gera Malha com resultados
Malha2D.Exporta_para_Gmsh(arquivo,IJ, XY, U, TVM,detJ)
Malha2D.AbreVisualizacaoGmsh(arquivo)

In [42]:
arquivo2 = 'ChapaCompleta.msh'

Materiais = {"MAT1":[1e9,0.3]}
apoios = {}
cargas = [("CARGA1",1,1000),("CARGA2",1,1000)]


Malha2D.Gerar_chapa_tracao_completa(nome_arquivo=arquivo2, L=1.0, D=0.1, h_fino=0.01, h_grosso=0.05)
nn, XY, ne, IJ, MAT, ESP, na, AP, nc, P = Malha2D.Processar_malha2D(arquivo2,Materiais,apoios,cargas,espessura=0.01)

In [43]:
#Monta Rigidez Global
K_b = BL4.RigidezGlobal(nn,ne,MAT,ESP,XY,IJ)
#Monta Força Global
F_b = BL4.ForcapGlobal(nn,ESP,XY,IJ,nc,P)
#Aplica as CChs homogêneas
K_mod,F_mod = BL4.AplicaCCH(nn,na,AP,K_b,F_b)

#Calcula o Deslocamento
U = np.linalg.solve(K_mod,F_mod)
print(U)
sp.pprint(sp.latex(sp.Matrix(U)))

UX = U[0::2]
UY = U[1::2]


#Calcula a Tensão ao longo dos elementos
Sigma = BL4.CalculaTensaoMalha(nn,ne,MAT,ESP,XY,IJ,U)

detJ = BL4.CalculaDetJ(ne,XY,IJ)

#Calcula Von Mises
TVM = BL4.TensaoVonMises2D(ne,Sigma)



[[ 1.08598133e-07]
 [ 3.01042554e-08]
 [-4.63198662e-08]
 ...
 [ 4.62882303e-08]
 [-4.72289051e-07]
 [-1.19938705e-07]]
\left[\begin{matrix}1.08598132870849 \cdot 10^{-7}\\3.01042553920374 \cdot 10^ ↪

↪ {-8}\\-4.63198662466146 \cdot 10^{-8}\\-2.5059236139582 \cdot 10^{-8}\\-1.95 ↪

↪ 813747657783 \cdot 10^{-7}\\2.34527525368215 \cdot 10^{-8}\\-4.0734496880992 ↪

↪ 5 \cdot 10^{-8}\\7.85045271950756 \cdot 10^{-8}\\-5.08451084357309 \cdot 10^ ↪

↪ {-7}\\1.35540376719468 \cdot 10^{-7}\\4.81226642616379 \cdot 10^{-7}\\1.9530 ↪

↪ 5301269574 \cdot 10^{-7}\\4.21470608746775 \cdot 10^{-7}\\-8.20954011212309  ↪

↪ \cdot 10^{-8}\\-5.68222885700986 \cdot 10^{-7}\\-1.41923060409839 \cdot 10^{ ↪

↪ -7}\\1.04882911134299 \cdot 10^{-7}\\2.00455924144622 \cdot 10^{-8}\\9.53520 ↪

↪ 821361631 \cdot 10^{-8}\\1.01488247426725 \cdot 10^{-8}\\8.05906754592029 \c ↪

↪ dot 10^{-8}\\7.11530507340494 \cdot 10^{-10}\\6.14318964677118 \cdot 10^{-8} ↪

↪ \\-7.46882994530503 \cdot 10^{-9}\\3.83627239654267 \cdot 

In [44]:
Malha2D.Exporta_para_Gmsh(arquivo2,IJ, XY, U, TVM,detJ)
Malha2D.AbreVisualizacaoGmsh(arquivo2)